# Task 4: Sequential Pattern Mining

In [130]:
import pandas as pd
import copy

## Loading the new dataset

In [131]:
df = pd.read_csv('../dataset/new_customer_supermarket.csv', sep='\t', index_col=0)
df

,BasketID,BasketDate,Sale,CustomerID,ProdID,ProdDescr,Qta,TotSale
0,539993,2011-04-01 10:00:00,1.95,13313.0,22386,JUMBO BAG PINK POLKADOT,10,19.50
1,539993,2011-04-01 10:00:00,0.42,13313.0,21499,BLUE POLKADOT WRAP,25,10.50
2,539993,2011-04-01 10:00:00,0.42,13313.0,21498,RED RETROSPOT WRAP,25,10.50
3,539993,2011-04-01 10:00:00,2.10,13313.0,22379,RECYCLING BAG RETROSPOT,5,10.50
4,539993,2011-04-01 10:00:00,1.25,13313.0,20718,RED RETROSPOT SHOPPER BAG,10,12.50
...,...,...,...,...,...,...,...,...
363572,581587,2011-09-12 12:50:00,0.85,12680.0,22613,PACK OF SPACEBOY NAPKINS,12,10.20
363573,581587,2011-09-12 12:50:00,2.10,12680.0,22899,CHILDRENS APRON DOLLY GIRL,6,12.60
363574,581587,2011-09-12 12:50:00,4.15,12680.0,23254,CHILDRENS CUTLERY DOLLY GIRL,4,16.60
363575,581587,2011-09-12 12:50:00,4.15,12680.0,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,16.60


In [132]:
df.dtypes

BasketID        int64
BasketDate     object
Sale          float64
CustomerID    float64
ProdID         object
ProdDescr      object
Qta             int64
TotSale       float64
dtype: object

In [133]:
df = df.astype({'BasketDate': 'datetime64',
                'BasketID': 'object',
                'CustomerID': 'object'})

In [134]:
df['ProdDescr'].nunique()

3678

In [135]:
import webcolors
from nltk.corpus import stopwords
from nltk import word_tokenize, pos_tag
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))
colors = webcolors.CSS3_NAMES_TO_HEX.keys()

def lemmatize_descr(descr):
    return ' '.join(lemmatizer.lemmatize(token) for token in word_tokenize(descr.lower()) 
                    if not token in stop_words and 
                    not any(color in token for color in colors) and 
                    len(token) >= 3)

In [136]:
df['ProdDescr'] = df['ProdDescr'].apply(lemmatize_descr)
df

,BasketID,BasketDate,Sale,CustomerID,ProdID,ProdDescr,Qta,TotSale
0,539993,2011-04-01 10:00:00,1.95,13313,22386,jumbo bag polkadot,10,19.50
1,539993,2011-04-01 10:00:00,0.42,13313,21499,polkadot wrap,25,10.50
2,539993,2011-04-01 10:00:00,0.42,13313,21498,retrospot wrap,25,10.50
3,539993,2011-04-01 10:00:00,2.10,13313,22379,recycling bag retrospot,5,10.50
4,539993,2011-04-01 10:00:00,1.25,13313,20718,retrospot shopper bag,10,12.50
...,...,...,...,...,...,...,...,...
363572,581587,2011-09-12 12:50:00,0.85,12680,22613,pack spaceboy napkin,12,10.20
363573,581587,2011-09-12 12:50:00,2.10,12680,22899,childrens apron dolly girl,6,12.60
363574,581587,2011-09-12 12:50:00,4.15,12680,23254,childrens cutlery dolly girl,4,16.60
363575,581587,2011-09-12 12:50:00,4.15,12680,23255,childrens cutlery circus parade,4,16.60


In [137]:
df['ProdDescr'].nunique()

3259

## GSP: Apriori-based Sequential Pattern Mining

In [138]:
# An event is a list of strings.
# A sequence is a list of events.
# A dataset is a list of sequences.
# Thus, a dataset is a list of lists of lists of strings.

dataset = [
    [['a'], ['a', 'b', 'c'], ['a', 'c'], ['c']],
    [['a'], ['c'], ['b', 'c']],
    [['a', 'b'], ['d'], ['c'], ['b'], ['c']],
    [['a'], ['c'], ['b'], ['c']]
]

In [139]:
"""
This is a simple recursive method that checks if subsequence is a subSequence of mainSequence
"""
def isSubsequence(mainSequence, subSequence):
    subSequenceClone = list(subSequence) # clone the sequence, because we will alter it
    return isSubsequenceRecursive(mainSequence, subSequenceClone) # start recursion

"""
Function for the recursive call of isSubsequence, not intended for external calls
"""
def isSubsequenceRecursive(mainSequence, subSequenceClone, start=0):
    # check if empty: end of recursion, all itemsets have been found
    if not subSequenceClone:
        return True
    # retrieves element of the subsequence and removes is from subsequence 
    firstElem = set(subSequenceClone.pop(0))
    # search for the first itemset...
    for i in range(start, len(mainSequence)):
        if set(mainSequence[i]).issuperset(firstElem):
            # and recurse
            return isSubsequenceRecursive(mainSequence, subSequenceClone, i + 1)
    return False

In [140]:
aSequence = [['a'], ['b', 'c'], ['d'], ['a', 'e']]

In [141]:
isSubsequence(aSequence, [['a'], ['d'], ['e']])

True

In [142]:
isSubsequence(aSequence, [['a'], ['b', 'c'], ['e']])

True

In [143]:
isSubsequence(aSequence, [['a'], ['b', 'd']])

False

In [144]:
"""
Computes the length of the sequence (sum of the length of the contained itemsets)
"""
def sequenceLength(sequence):
    return sum(len(i) for i in sequence)

In [145]:
print(sequenceLength([['a'], ['b', 'c'], ['a'], ['b', 'c', 'd']]))

7


In [146]:
"""
Computes the support of a sequence in a dataset
"""
def countSupport(dataset, candidateSequence):
    return sum(1 for seq in dataset if isSubsequence(seq, candidateSequence))

In [147]:
countSupport(dataset, [['b']])

4

In [148]:
countSupport(dataset, [['a'], ['b', 'c']])

2

In [149]:
"""
Generates one candidate of length k from two candidates of length (k-1) as used in the GSP algorithm
"""
def generateCandidatesForPair(cand1, cand2):
    cand1Clone = copy.deepcopy(cand1)
    cand2Clone = copy.deepcopy(cand2)
    # drop the leftmost item from cand1:
    if len(cand1[0]) == 1:
        cand1Clone.pop(0)
    else:
        cand1Clone[0] = cand1Clone[0][1:]
    # drop the rightmost item from cand2:
    if len(cand2[-1]) == 1:
        cand2Clone.pop(-1)
    else:
        cand2Clone[-1] = cand2Clone[-1][:-1]
    
    # if the result is not the same, then we dont need to join
    if not cand1Clone == cand2Clone:
        return []
    else:
        newCandidate = copy.deepcopy(cand1)
        if len(cand2[-1]) == 1:
            newCandidate.append(cand2[-1])
        else:
            newCandidate[-1].extend(cand2[-1][-1])
        return newCandidate

In [150]:
candidateA = [['a'], ['b', 'c'], ['d']]
candidateB = [['b', 'c'], ['d', 'e']]
generateCandidatesForPair(candidateA, candidateB)

[['a'], ['b', 'c'], ['d', 'e']]

In [151]:
candidateA = [['a'], ['b', 'c'], ['d']]
candidateC = [['b', 'c'], ['d'], ['e']]
generateCandidatesForPair(candidateA, candidateC)

[['a'], ['b', 'c'], ['d'], ['e']]

In [152]:
candidateA = [['a'], ['b', 'c'], ['d']]
candidateD = [['a'], ['b', 'c'], ['e']]
generateCandidatesForPair(candidateA, candidateD)

[]

In [153]:
"""
Generates the set of candidates of length k from the set of frequent sequences with length (k-1)
"""
def generateCandidates(lastLevelCandidates):
    k = sequenceLength(lastLevelCandidates[0]) + 1
    if k == 2:
        flatShortCandidates = [item for sublist2 in lastLevelCandidates for sublist1 in sublist2 for item in sublist1]
        result = [[[a, b]] for a in flatShortCandidates for b in flatShortCandidates if b > a]
        result.extend([[[a], [b]] for a in flatShortCandidates for b in flatShortCandidates])
        return result
    else:
        candidates = []
        for i in range(0, len(lastLevelCandidates)):
            for j in range(0, len(lastLevelCandidates)):
                newCand = generateCandidatesForPair(lastLevelCandidates[i], lastLevelCandidates[j])
                if not newCand == []:
                    candidates.append(newCand)
        candidates.sort()
        return candidates

An example; lets assume, we know the frequent sequences of level 2:

In [154]:
lastLevelFrequentPatterns = [
    [['a', 'b']], 
    [['b', 'c']], 
    [['a'], ['b']], 
    [['a'], ['c']], 
    [['b'], ['c']], 
    [['c'], ['b']], 
    [['c'], ['c']]
]

Then we can compute the generate candidates for level 3

In [155]:
newCandidates = generateCandidates(lastLevelFrequentPatterns)
newCandidates

[[['a'], ['b'], ['c']],
 [['a'], ['b', 'c']],
 [['a'], ['c'], ['b']],
 [['a'], ['c'], ['c']],
 [['a', 'b'], ['c']],
 [['a', 'b', 'c']],
 [['b'], ['c'], ['b']],
 [['b'], ['c'], ['c']],
 [['b', 'c'], ['b']],
 [['b', 'c'], ['c']],
 [['c'], ['b'], ['c']],
 [['c'], ['b', 'c']],
 [['c'], ['c'], ['b']],
 [['c'], ['c'], ['c']]]

In [156]:
"""
Computes all direct subsequence for a given sequence.
A direct subsequence is any sequence that originates from deleting exactly one item from any event in the original sequence.
"""
def generateDirectSubsequences(sequence):
    result = []
    for i, itemset in enumerate(sequence):
        if len(itemset) == 1:
            sequenceClone = copy.deepcopy(sequence)
            sequenceClone.pop(i)
            result.append(sequenceClone)
        else:
            for j in range(len(itemset)):
                sequenceClone = copy.deepcopy(sequence)
                sequenceClone[i].pop(j)
                result.append(sequenceClone)
    return result

In [157]:
"""
Prunes the set of candidates generated for length k given all frequent sequence of level (k-1), as done in GSP
"""
def pruneCandidates(candidatesLastLevel, candidatesGenerated):
    return [cand for cand in candidatesGenerated if all(x in candidatesLastLevel for x in generateDirectSubsequences(cand))]

We apply this on example dataset:

In [158]:
candidatesPruned = pruneCandidates(lastLevelFrequentPatterns, newCandidates)
candidatesPruned

[[['a'], ['b'], ['c']],
 [['a'], ['b', 'c']],
 [['a'], ['c'], ['b']],
 [['a'], ['c'], ['c']],
 [['a', 'b'], ['c']],
 [['b'], ['c'], ['c']],
 [['b', 'c'], ['c']],
 [['c'], ['b'], ['c']],
 [['c'], ['b', 'c']],
 [['c'], ['c'], ['b']],
 [['c'], ['c'], ['c']]]

In [159]:
minSupport = 2
candidatesCounts = [(i, countSupport(dataset, i)) for i in candidatesPruned]
resultLvl = [(i, count) for i, count in candidatesCounts if count >= minSupport]
resultLvl

[([['a'], ['b'], ['c']], 3),
 ([['a'], ['b', 'c']], 2),
 ([['a'], ['c'], ['b']], 3),
 ([['a'], ['c'], ['c']], 4),
 ([['a', 'b'], ['c']], 2),
 ([['b'], ['c'], ['c']], 2),
 ([['c'], ['b'], ['c']], 2)]

In [160]:
"""
The GSP algorithm: Apriori-based Sequential Pattern Mining. Computes the frequent sequences in a seqeunce dataset for a given minSupport.

Args:
    dataset: a list of sequences, for which the frequent (sub-)sequences are computed
    minSupport: the minimum support that makes a sequence frequent
    verbose: if true, additional information on the mining process is printed (i.e., candidates on each level)

Returns:
    A list of tuples (s, c), where s is a frequent sequence, and c is the count for that sequence
"""
def gsp(dataset, minSupport, verbose=False):
    global numberOfCountingOperations
    numberOfCountingOperations = 0
    overall = []
    itemsInDataset = sorted(set([item for sublist1 in dataset for sublist2 in sublist1 for item in sublist2]))
    singleItemSequences = [[[item]] for item in itemsInDataset]
    singleItemCounts = [(i, countSupport(dataset, i)) for i in singleItemSequences if countSupport(dataset, i) >= minSupport]
    overall.append(singleItemCounts)
    print('Result, lvl 1: ' + str(overall[0]))
    k = 1
    while True:
        if not overall[k - 1]:
            break
        # 1. candidate generation
        candidatesLastLevel = [x[0] for x in overall[k - 1]]
        candidatesGenerated = generateCandidates(candidatesLastLevel)
        # 2. candidate pruning (using a "containsall" subsequences)
        candidatesPruned = [cand for cand in candidatesGenerated if all(x in candidatesLastLevel for x in generateDirectSubsequences(cand))]
        # 3. candidate checking
        candidatesCounts = [(i, countSupport(dataset, i)) for i in candidatesPruned]
        resultLvl = [(i, count) for i, count in candidatesCounts if count >= minSupport]
        if verbose:
            print('Candidates generated, lvl ' + str(k + 1) + ': ' + str(candidatesGenerated))
            print('Candidates pruned, lvl ' + str(k + 1) + ': ' + str(candidatesPruned))
            print('Result, lvl ' + str(k + 1) + ': ' + str(resultLvl))
        overall.append(resultLvl)
        k = k + 1
    # "flatten" overall
    overall = overall[:-1]
    overall = [item for sublist in overall for item in sublist]
    return overall

In [161]:
gsp(dataset, minSupport=2, verbose=False)

Result, lvl 1: [([['a']], 4), ([['b']], 4), ([['c']], 4)]


[([['a']], 4),
 ([['b']], 4),
 ([['c']], 4),
 ([['a', 'b']], 2),
 ([['b', 'c']], 2),
 ([['a'], ['b']], 4),
 ([['a'], ['c']], 4),
 ([['b'], ['c']], 3),
 ([['c'], ['b']], 3),
 ([['c'], ['c']], 4),
 ([['a'], ['b'], ['c']], 3),
 ([['a'], ['b', 'c']], 2),
 ([['a'], ['c'], ['b']], 3),
 ([['a'], ['c'], ['c']], 4),
 ([['a', 'b'], ['c']], 2),
 ([['b'], ['c'], ['c']], 2),
 ([['c'], ['b'], ['c']], 2),
 ([['a'], ['c'], ['b'], ['c']], 2),
 ([['a', 'b'], ['c'], ['c']], 2)]

### Supermarket dataset

In [162]:
grouped = pd.DataFrame(df.groupby(['CustomerID', 'BasketID'])['ProdDescr'].apply(list))
grouped

ProdDescr
CustomerID BasketID                                                   
12347.0    542237    [new baroquecandlestick candle, new baroque ca...
           549222    [airline bag vintage jet set, airline bag vint...
           556201    [rabbit night light, regency tea strainer, reg...
           562032    [set vintage leaf cake case, set heart shape p...
           573511    [mini light woodland mushroom, goose feather t...
...                                                                ...
18283.0    579673    [set snack loaf baking case, colour spaceboy p...
           580872    [feltcraft hairband, pack dolly girl tissue, p...
18287.0    554065    [small babushka notebook, small babushka noteb...
           570715    [hand warmer owl design, set wooden sleigh dec...
           573167    [set wooden sleigh decoration, painted metal s...

[16921 rows x 1 columns]

In [163]:
grouped = pd.DataFrame(grouped.groupby('CustomerID')['ProdDescr'].apply(list))
grouped

,ProdDescr
CustomerID,
12347.0,"[[new baroquecandlestick candle, new baroque c..."
12348.0,"[[pack retrospot tissue, pack heart design tis..."
12349.0,"[[parisienne curio cabinet, sweetheart wall ti..."
12350.0,"[[way metal sign, metal sign neighbourhood wit..."
12352.0,"[[wooden happy birthday garland, doughnut trin..."
...,...
18280.0,"[[wood board ant finish, retrospot lamp, gumba..."
18281.0,"[[robot birthday card, card circus parade, pen..."
18282.0,"[[antique cream cutlery cupboard, french style..."


In [166]:
supermarket = grouped['ProdDescr'].tolist()

In [ ]:
gsp(supermarket, minSupport=2, verbose=False)

Result, lvl 1: [([['abc treasure book box']], 157), ([['abstract circle journal']], 21), ([['abstract circle pocket book']], 34), ([['abstract circle sketchbook']], 40), ([['acrylic faceted bangle']], 4), ([['acrylic geometric lamp']], 8), ([['acrylic hanging']], 22), ([['acrylic jewel']], 7), ([['acrylic jewel icicle']], 17), ([['adult apron apple delight']], 32), ([['advent calendar gingham sack']], 121), ([['afghan slipper sock pair']], 10), ([['aged glass tlight holder']], 146), ([['airline bag vintage jet set']], 128), ([['airline bag vintage tokyo']], 129), ([['airline bag vintage world champion']], 77), ([['airline loungemetal sign']], 83), ([['alarm clock bakelike']], 496), ([['allium artificial flower']], 3), ([['alphabet heart sticker sheet']], 23), ([['alphabet stencil craft']], 110), ([['alphabet wall art']], 14), ([['aluminium heart']], 6), ([['aluminium stamped heart']], 100), ([['amber bracelet']], 7), ([['amber chunky bead bracelet strap']], 3), ([['amber chunky glassbe